In [2]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
import math
from variable_functions import *

In [3]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [4]:
with open('../../../filelists/800to1000_files.txt', 'r') as f:
    files = [line.strip() for line in f]

In [5]:
files[10]

'nano_mc2017_1-108.root'

In [64]:
uncut_events = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/qcd/800to1000/' + files[10]: "/Events"},
    delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [145]:
fatjetSelect = (
            (events.FatJet.pt > 250)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.5)
            & (events.FatJet.mass > 50)
            & (events.FatJet.mass < 200)
        )
events = uncut_events[
    ak.any(fatjetSelect, axis=1)
]

cut = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut, axis=1)]

In [148]:
events['FatJet', 'd2b1'] = ak.unflatten(
     d2_calc(events.FatJet), counts=ak.num(events.FatJet)
)